# 🔒 Sistema di Anonimizzazione con NER e Grafi NetworkX
# 
# 📋 Panoramica
# 
# Questo notebook implementa un sistema avanzato di anonimizzazione che combina:
# - **🔍 Named Entity Recognition (NER)** per identificare entità semantiche
# - **📝 Espressioni Regolari** per pattern specifici italiani
# - **🕸️ Grafi NetworkX** per visualizzare le relazioni tra entità
# 
# 🎯 Obiettivi
# 1. Identificare e mascherare informazioni sensibili (PII)
# 2. Creare un grafo delle relazioni tra entità
# 3. Visualizzare i risultati in modo interattivo
# 
# 🛠️ Tecnologie utilizzate
# - `transformers` - Modelli NER pre-addestrati
# - `networkx` - Creazione e manipolazione di grafi
# - `matplotlib` - Visualizzazione dei grafi
# - `streamlit` - Interfaccia utente (opzionale)

In [2]:
# Installa le dipendenze necessarie
import subprocess
import sys

def install_requirements():
    """Installa le dipendenze necessarie"""
    requirements = [
        'transformers',
        'torch',
        'networkx',
        'matplotlib',
        'streamlit',
        'pandas',
        'numpy'
    ]
    
    for package in requirements:
        try:
            __import__(package)
            print(f"✅ {package} già installato")
        except ImportError:
            print(f"📦 Installazione {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} installato con successo")

In [3]:
# Importazioni principali
import re
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from typing import Dict, Tuple, List, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurazione matplotlib per grafici più belli
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

In [5]:
# Configurazione principale
class Config:
    """Configurazioni del sistema di anonimizzazione"""
    
    # Modello NER (scegli in base alle tue esigenze)
    NER_MODEL = "dbmdz/bert-large-cased-finetuned-conll03-english"
    # Alternativa per italiano: "dbmdz/bert-base-italian-cased-ner"
    
    # Soglia di confidenza per NER
    NER_CONFIDENCE_THRESHOLD = 0.5
    
    # Impostazioni visualizzazione
    FIGURE_SIZE = (14, 10)
    DPI = 300

# Pattern regex per entità italiane
REGEX_PATTERNS = {
    "CODICE_FISCALE": r'\b[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]\b',
    "PARTITA_IVA": r'\b\d{11}\b',
    "EMAIL": r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b',
    "TELEFONO": r'\b(?:\+39\s?)?(?:0\d{1,4}[-.\s]?)?\d{6,8}\b',
    "DATA": r'\b(?:0[1-9]|[12][0-9]|3[01])[\/\-\.](0[1-9]|1[012])[\/\-\.](?:19|20)\d{2}\b',
    "IBAN": r'\b[A-Z]{2}\d{2}[A-Z0-9]{4}\d{7}[A-Z0-9]{0,16}\b',
    "IMPORTO": r'\b\d{1,3}(?:\.\d{3})*(?:,\d{2})?\s?€|\b€\s?\d{1,3}(?:\.\d{3})*(?:,\d{2})?\b',
    "INDIRIZZO": r'\b(?:Via|Piazza|Viale|Corso|Largo|Vicolo)\s+[A-Za-z\s]+\d+\b',
    "CAP": r'\b\d{5}\b',
    "CARTA_CREDITO": r'\b(?:\d{4}[-.\s]?){3}\d{4}\b',
}

# Colori per la visualizzazione
ENTITY_COLORS = {
    'PERSONA': '#FF6B6B',      # Rosso corallo
    'LUOGO': '#4ECDC4',        # Turchese
    'ORGANIZZAZIONE': '#45B7D1', # Blu
    'EMAIL': '#96CEB4',        # Verde menta
    'TELEFONO': '#FFEAA7',     # Giallo
    'CODICE_FISCALE': '#DDA0DD', # Prugna
    'PARTITA_IVA': '#98D8C8',  # Acqua marina
    'DATA': '#F7DC6F',         # Giallo oro
    'IBAN': '#BB8FCE',         # Lavanda
    'CARTA_CREDITO': '#F1948A', # Salmone
    'INDIRIZZO': '#85C1E9',    # Blu cielo
    'IMPORTO': '#82E0AA',      # Verde chiaro
    'CAP': '#F8C471',          # Arancione
    'Documento': '#D5DBDB',    # Grigio chiaro
    'Categoria': '#AED6F1',    # Blu pastello
    'default': '#BDC3C7'       # Grigio
}

print("⚙️ Configurazione completata")
print(f"📝 {len(REGEX_PATTERNS)} pattern regex definiti")
print(f"🎨 {len(ENTITY_COLORS)} colori per entità configurati")


⚙️ Configurazione completata
📝 10 pattern regex definiti
🎨 16 colori per entità configurati


In [6]:
# %%
class NERAnonimizer:
    """🔒 Anonimizzatore avanzato con NER, regex e grafi"""
    
    def __init__(self, use_ner: bool = True, verbose: bool = True):
        """
        Inizializza l'anonimizzatore
        
        Args:
            use_ner: Se utilizzare il modello NER
            verbose: Se stampare messaggi di debug
        """
        self.regex_patterns = REGEX_PATTERNS
        self._ner_pipe = None
        self.use_ner = use_ner
        self.verbose = verbose
        
        # Inizializza il grafo diretto
        self.G = nx.DiGraph()
        
        # Statistiche
        self.stats = {
            'documenti_processati': 0,
            'entità_trovate': 0,
            'tempo_elaborazione': 0
        }
        
        if self.verbose:
            print("🚀 NERAnonimizer inizializzato")
            print(f"🧠 NER: {'Attivo' if use_ner else 'Disattivo'}")
            print(f"📊 Grafo inizializzato: {self.G.number_of_nodes()} nodi")
    
    @property
    def ner_pipe(self):
        """Lazy loading del modello NER"""
        if not self.use_ner:
            return None
            
        if self._ner_pipe is None:
            if self.verbose:
                print("📥 Caricamento modello NER...")
            
            try:
                from transformers import pipeline
                self._ner_pipe = pipeline(
                    "ner",
                    model=Config.NER_MODEL,
                    aggregation_strategy="simple"
                )
                if self.verbose:
                    print("✅ Modello NER caricato con successo")
            except Exception as e:
                if self.verbose:
                    print(f"❌ Errore caricamento NER: {e}")
                    print("🔄 Continuando solo con regex...")
                return None
        
        return self._ner_pipe
    
    def add_entity_to_graph(self, entity_id: str, entity_type: str, original_text: str):
        """Aggiunge un'entità al grafo"""
        # Aggiungi nodo per l'entità
        self.G.add_node(entity_id, 
                       tipo=entity_type, 
                       testo_originale=original_text,
                       timestamp=datetime.now().isoformat())
        
        # Aggiungi nodo per il tipo se non esiste
        if not self.G.has_node(entity_type):
            self.G.add_node(entity_type, tipo="Categoria")
        
        # Collega l'entità al suo tipo
        self.G.add_edge(entity_id, entity_type, relazione="è_di_tipo")
    
    def add_document_to_graph(self, doc_id: str, entities: Dict):
        """Aggiunge un documento e le sue entità al grafo"""
        # Aggiungi nodo documento
        self.G.add_node(doc_id, 
                       tipo="Documento",
                       num_entità=len(entities),
                       timestamp=datetime.now().isoformat())
        
        # Collega documento alle entità trovate
        for entity_id in entities.keys():
            if self.G.has_node(entity_id):
                self.G.add_edge(doc_id, entity_id, relazione="contiene")
    
    def mask_with_regex(self, text: str) -> Tuple[str, Dict]:
        """🔍 Applica mascheramento con espressioni regolari"""
        masked_text = text
        found_entities = {}
        
        # Ordina pattern per lunghezza (più lunghi prima)
        sorted_patterns = sorted(
            self.regex_patterns.items(),
            key=lambda item: len(item[1]),
            reverse=True
        )
        
        for label, pattern in sorted_patterns:
            matches = list(re.finditer(pattern, masked_text, flags=re.IGNORECASE))
            for match in reversed(matches):
                original = match.group()
                if original.startswith('[') and original.endswith(']'):
                    continue
                
                placeholder = f"[{label}_{len(found_entities)}]"
                found_entities[placeholder] = original
                
                # Aggiungi al grafo
                self.add_entity_to_graph(placeholder, label, original)
                
                masked_text = masked_text[:match.start()] + placeholder + masked_text[match.end():]
        
        return masked_text, found_entities
    
    def mask_with_ner(self, text: str) -> Tuple[str, Dict]:
        """🧠 Applica mascheramento con NER"""
        if not self.ner_pipe:
            return text, {}
        
        try:
            entities = self.ner_pipe(text)
            entity_map = {}
            
            sorted_entities = sorted(entities, key=lambda x: x['start'], reverse=True)
            
            for ent in sorted_entities:
                if ent['score'] > Config.NER_CONFIDENCE_THRESHOLD:
                    label = ent['entity_group']
                    original_text = text[ent['start']:ent['end']]
                    
                    if original_text.startswith('[') and original_text.endswith(']'):
                        continue
                    
                    placeholder = f"[{label}_{len(entity_map)}]"
                    entity_map[placeholder] = original_text
                    
                    # Aggiungi al grafo
                    self.add_entity_to_graph(placeholder, label, original_text)
                    
                    text = text[:ent['start']] + placeholder + text[ent['end']:]
            
            return text, entity_map
        
        except Exception as e:
            if self.verbose:
                print(f"❌ Errore NER: {e}")
            return text, {}
    
    def anonymize(self, text: str, doc_id: str = None) -> Tuple[str, Dict]:
        """🔒 Pipeline completa di anonimizzazione"""
        if not text or not text.strip():
            return text, {}
        
        start_time = datetime.now()
        
        # Regex prima, poi NER
        masked_text, regex_entities = self.mask_with_regex(text)
        final_text, ner_entities = self.mask_with_ner(masked_text)
        
        # Combina entità
        all_entities = {**regex_entities, **ner_entities}
        
        # Aggiungi documento al grafo se specificato
        if doc_id:
            self.add_document_to_graph(doc_id, all_entities)
        
        # Aggiorna statistiche
        self.stats['documenti_processati'] += 1
        self.stats['entità_trovate'] += len(all_entities)
        self.stats['tempo_elaborazione'] += (datetime.now() - start_time).total_seconds()
        
        if self.verbose:
            print(f"✅ Processato documento: {len(all_entities)} entità trovate")
        
        return final_text, all_entities
    
    def get_graph_stats(self) -> Dict:
        """📊 Statistiche del grafo"""
        return {
            "numero_nodi": self.G.number_of_nodes(),
            "numero_collegamenti": self.G.number_of_edges(),
            "tipi_entità": list(set([data.get('tipo', 'Sconosciuto') 
                                   for node, data in self.G.nodes(data=True)])),
            "entità_per_tipo": {tipo: len([n for n, d in self.G.nodes(data=True) 
                                         if d.get('tipo') == tipo]) 
                               for tipo in set([data.get('tipo', 'Sconosciuto') 
                                              for node, data in self.G.nodes(data=True)])}
        }
    
    def visualize_graph(self, 
                       layout_type: str = 'spring',
                       figsize: Tuple[int, int] = (14, 10),
                       save_path: Optional[str] = None,
                       show_labels: bool = True,
                       title: str = "🕸️ Grafo delle Entità Estratte") -> None:
        """
        🎨 Visualizza il grafo con stile avanzato
        
        Args:
            layout_type: Tipo di layout ('spring', 'circular', 'kamada_kawai')
            figsize: Dimensioni della figura
            save_path: Percorso per salvare l'immagine
            show_labels: Se mostrare le etichette
            title: Titolo del grafo
        """
        if self.G.number_of_nodes() == 0:
            print("⚠️ Il grafo è vuoto!")
            return
        
        # Crea figura con subplot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, gridspec_kw={'width_ratios': [3, 1]})
        
        # === GRAFO PRINCIPALE ===
        
        # Crea il layout
        layouts = {
            'spring': lambda: nx.spring_layout(self.G, k=3, iterations=50),
            'circular': lambda: nx.circular_layout(self.G),
            'kamada_kawai': lambda: nx.kamada_kawai_layout(self.G),
            'random': lambda: nx.random_layout(self.G)
        }
        
        pos = layouts.get(layout_type, layouts['spring'])()
        
        # Prepara colori e dimensioni dei nodi
        node_colors = []
        node_sizes = []
        node_types = []
        
        for node, data in self.G.nodes(data=True):
            node_type = data.get('tipo', 'default')
            node_types.append(node_type)
            node_colors.append(ENTITY_COLORS.get(node_type, ENTITY_COLORS['default']))
            
            # Dimensioni diverse per tipi diversi
            if node_type == 'Documento':
                node_sizes.append(2500)
            elif node_type == 'Categoria':
                node_sizes.append(1800)
            else:
                node_sizes.append(1200)
        
        # Disegna i nodi
        nx.draw_networkx_nodes(self.G, pos, 
                              node_color=node_colors,
                              node_size=node_sizes,
                              alpha=0.8,
                              edgecolors='white',
                              linewidths=2,
                              ax=ax1)
        
        # Disegna gli archi con stili diversi
        edge_colors = []
        edge_styles = []
        for edge in self.G.edges(data=True):
            relation = edge[2].get('relazione', 'default')
            if relation == 'contiene':
                edge_colors.append('#2C3E50')
                edge_styles.append('solid')
            else:
                edge_colors.append('#7F8C8D')
                edge_styles.append('dashed')
        
        nx.draw_networkx_edges(self.G, pos,
                              edge_color=edge_colors,
                              arrows=True,
                              arrowsize=25,
                              alpha=0.7,
                              width=2,
                              ax=ax1)
        
        # Etichette se richieste
        if show_labels:
            labels = {}
            for node, data in self.G.nodes(data=True):
                node_type = data.get('tipo', 'Unknown')
                if node_type in ['Categoria', 'Documento']:
                    labels[node] = node
                else:
                    labels[node] = node_type
            
            nx.draw_networkx_labels(self.G, pos, labels, 
                                   font_size=9, 
                                   font_weight='bold',
                                   font_color='white',
                                   ax=ax1)
        
        ax1.set_title(title, fontsize=16, fontweight='bold', pad=20)
        ax1.axis('off')
        
        # === STATISTICHE E LEGENDA ===
        
        stats = self.get_graph_stats()
        
        # Prepara dati per il grafico a barre
        types_in_graph = [t for t in stats['entità_per_tipo'].keys() if t != 'Sconosciuto']
        counts = [stats['entità_per_tipo'][t] for t in types_in_graph]
        colors_bar = [ENTITY_COLORS.get(t, ENTITY_COLORS['default']) for t in types_in_graph]
        
        # Grafico a barre orizzontale
        y_pos = np.arange(len(types_in_graph))
        bars = ax2.barh(y_pos, counts, color=colors_bar, alpha=0.8, edgecolor='white')
        
        # Personalizza il grafico delle statistiche
        ax2.set_yticks(y_pos)
        ax2.set_yticklabels(types_in_graph, fontsize=9)
        ax2.set_xlabel('Numero di Entità', fontsize=10)
        ax2.set_title('📊 Distribuzione Entità', fontsize=12, fontweight='bold')
        ax2.grid(axis='x', alpha=0.3)
        
        # Aggiungi valori sulle barre
        for i, (bar, count) in enumerate(zip(bars, counts)):
            ax2.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
                    str(count), ha='left', va='center', fontweight='bold')
        
        # Informazioni generali
        info_text = f"""
📊 STATISTICHE GENERALI
────────────────────────
🔗 Nodi totali: {stats['numero_nodi']}
🔗 Collegamenti: {stats['numero_collegamenti']}
📄 Documenti: {self.stats['documenti_processati']}
🎯 Entità trovate: {self.stats['entità_trovate']}
⏱️ Tempo elaborazione: {self.stats['tempo_elaborazione']:.2f}s
        """
        
        ax2.text(0.02, 0.02, info_text, transform=ax2.transAxes, 
                fontsize=8, verticalalignment='bottom',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgray', alpha=0.8))
        
        plt.tight_layout()
        
        # Salva se richiesto
        if save_path:
            plt.savefig(save_path, dpi=Config.DPI, bbox_inches='tight', 
                       facecolor='white', edgecolor='none')
            print(f"💾 Grafo salvato in: {save_path}")
        
        plt.show()
    
    def print_detailed_summary(self):
        """📋 Stampa un riassunto dettagliato del sistema"""
        print("\n" + "="*70)
        print("🔒 SISTEMA DI ANONIMIZZAZIONE - REPORT DETTAGLIATO")
        print("="*70)
        
        stats = self.get_graph_stats()
        
        print(f"\n📊 STATISTICHE GENERALI")
        print("-" * 30)
        print(f"🔗 Nodi nel grafo: {stats['numero_nodi']}")
        print(f"🔗 Collegamenti: {stats['numero_collegamenti']}")
        print(f"📄 Documenti processati: {self.stats['documenti_processati']}")
        print(f"🎯 Entità totali trovate: {self.stats['entità_trovate']}")
        print(f"⏱️ Tempo elaborazione: {self.stats['tempo_elaborazione']:.2f} secondi")
        
        print(f"\n📋 DISTRIBUZIONE PER TIPO")
        print("-" * 30)
        for tipo, count in sorted(stats['entità_per_tipo'].items(), key=lambda x: x[1], reverse=True):
            emoji = "📄" if tipo == "Documento" else "📂" if tipo == "Categoria" else "🎯"
            print(f"{emoji} {tipo}: {count}")
        
        print(f"\n📄 DETTAGLI ENTITÀ")
        print("-" * 30)
        for node, data in self.G.nodes(data=True):
            tipo = data.get('tipo', 'Sconosciuto')
            testo = data.get('testo_originale', 'N/A')
            
            # Tronca il testo se troppo lungo
            if len(testo) > 40:
                testo = testo[:40] + "..."
            
            timestamp = data.get('timestamp', 'N/A')
            print(f"   🔸 {node} ({tipo})")
            print(f"      📝 Testo: {testo}")
            if timestamp != 'N/A':
                print(f"      ⏰ Creato: {timestamp}")
    
    def export_results(self, filepath: str, format: str = 'json'):
        """💾 Esporta i risultati in vari formati"""
        if format == 'json':
            import json
            
            # Prepara dati per JSON
            export_data = {
                'timestamp': datetime.now().isoformat(),
                'statistics': self.get_graph_stats(),
                'system_stats': self.stats,
                'nodes': [
                    {
                        'id': node,
                        'type': data.get('tipo', 'Unknown'),
                        'original_text': data.get('testo_originale', 'N/A'),
                        'timestamp': data.get('timestamp', 'N/A')
                    }
                    for node, data in self.G.nodes(data=True)
                ],
                'edges': [
                    {
                        'source': edge[0],
                        'target': edge[1],
                        'relation': edge[2].get('relazione', 'unknown')
                    }
                    for edge in self.G.edges(data=True)
                ]
            }
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(export_data, f, indent=2, ensure_ascii=False)
            
            print(f"💾 Risultati esportati in JSON: {filepath}")
        
        elif format == 'csv':
            # Esporta entità in CSV
            entities_data = []
            for node, data in self.G.nodes(data=True):
                entities_data.append({
                    'ID': node,
                    'Tipo': data.get('tipo', 'Unknown'),
                    'Testo_Originale': data.get('testo_originale', 'N/A'),
                    'Timestamp': data.get('timestamp', 'N/A')
                })
            
            df = pd.DataFrame(entities_data)
            df.to_csv(filepath, index=False, encoding='utf-8')
            print(f"💾 Risultati esportati in CSV: {filepath}")
        
        elif format == 'gexf':
            # Esporta grafo in formato GEXF per Gephi
            nx.write_gexf(self.G, filepath)
            print(f"💾 Grafo esportato in GEXF: {filepath}")
    
    def clear_graph(self):
        """🧹 Pulisce il grafo e resetta le statistiche"""
        self.G.clear()
        self.stats = {
            'documenti_processati': 0,
            'entità_trovate': 0,
            'tempo_elaborazione': 0
        }
        if self.verbose:
            print("🧹 Grafo e statistiche resettati")

print("✅ Classe NERAnonimizer definita con successo!")

✅ Classe NERAnonimizer definita con successo!


In [7]:
### 📝 Esempio 1: Documento Semplice

# %%
# Inizializza l'anonimizzatore (solo regex per test rapido)
print("🚀 Inizializzazione sistema di anonimizzazione...")
anonimizer = NERAnonimizer(use_ner=False, verbose=True)

# Testo di esempio
testo_esempio_1 = """
Mario Rossi, nato il 15/03/1985, risiede in Via Roma 123, Milano.
Il suo numero di telefono è 02-1234567 e la sua email è mario.rossi@gmail.com.
Ha un codice fiscale RSSMRA85C15F205X e partita IVA 12345678901.
Il suo IBAN è IT60X0542811101000000123456 e paga sempre con la carta 4532-1234-5678-9012.
"""

print("\n📄 TESTO ORIGINALE:")
print("-" * 50)
print(testo_esempio_1)

# Anonimizza il documento
print("\n🔄 Processamento in corso...")
testo_anonimizzato, entità_trovate = anonimizer.anonymize(testo_esempio_1, doc_id="documento_esempio_1")

print("\n🔒 TESTO ANONIMIZZATO:")
print("-" * 50)
print(testo_anonimizzato)

print(f"\n🎯 ENTITÀ IDENTIFICATE ({len(entità_trovate)}):")
print("-" * 50)
for placeholder, valore_originale in entità_trovate.items():
    tipo_entità = placeholder.split('[')[1].split('_')[0]
    print(f"🔸 {tipo_entità}: {valore_originale} → {placeholder}")

# %% [markdown]
# ### 📊 Visualizzazione Risultati Esempio 1

# %%
# Mostra riassunto dettagliato
anonimizer.print_detailed_summary()

# Visualizza il grafo
print("\n🎨 Generazione visualizzazione grafo...")
anonimizer.visualize_graph(
    layout_type='spring',
    figsize=(16, 10),
    title="🔒 Grafo Anonimizzazione - Documento Esempio 1"
)

🚀 Inizializzazione sistema di anonimizzazione...
🚀 NERAnonimizer inizializzato
🧠 NER: Disattivo
📊 Grafo inizializzato: 0 nodi

📄 TESTO ORIGINALE:
--------------------------------------------------

Mario Rossi, nato il 15/03/1985, risiede in Via Roma 123, Milano.
Il suo numero di telefono è 02-1234567 e la sua email è mario.rossi@gmail.com.
Ha un codice fiscale RSSMRA85C15F205X e partita IVA 12345678901.
Il suo IBAN è IT60X0542811101000000123456 e paga sempre con la carta 4532-1234-5678-9012.


🔄 Processamento in corso...
✅ Processato documento: 8 entità trovate

🔒 TESTO ANONIMIZZATO:
--------------------------------------------------

Mario Rossi, nato il [DATA_0], risiede in [INDIRIZZO_1], Milano.
Il suo numero di telefono è [TELEFONO_4] e la sua email è [EMAIL_2].
Ha un codice fiscale [CODICE_FISCALE_5] e partita IVA [PARTITA_IVA_7].
Il suo IBAN è [IBAN_3] e paga sempre con la carta [CARTA_CREDITO_6].


🎯 ENTITÀ IDENTIFICATE (8):
--------------------------------------------------
🔸 

In [8]:
### 📝 Esempio 2: Documenti Multipli

# %%
# Aggiungi più documenti per testare le relazioni
documenti_esempio = {
    "fattura_001": """
    Fattura n. 2024/001 del 12/01/2024
    Cliente: Giulia Verdi - Via Garibaldi 45, Roma
    Email: giulia.verdi@email.it - Tel: 06-9876543
    Codice Fiscale: VRDGLI90A52H501Z
    Importo: 1.250,00€
    """,
    
    "contratto_002": """
    Contratto di lavoro - Marco Bianchi
    Nato il 22/08/1990, residente in Corso Italia 88, Torino
    Telefono: 011-555-1234, Email: m.bianchi@company.com
    Codice Fiscale: BNCMRC90M22L219K
    Stipendio: 2.500,00€ mensili
    """,
    
    "    preventivo_003": """
    Preventivo n. 2024-PRV-003
    Azienda: Tech Solutions SRL - P.IVA 98765432101
    Via Montenapoleone 12, Milano - Tel: 02-1111222
    Email: info@techsolutions.it
    Importo totale: 15.750,00€
    IBAN: IT75B0300203280123456789012
    """
}

print("📚 Processamento documenti multipli...")
print("=" * 60)

# Processa tutti i documenti
risultati_multipli = {}
for doc_id, contenuto in documenti_esempio.items():
    print(f"\n📄 Processando: {doc_id}")
    print("-" * 40)
    
    testo_anonimizzato, entità = anonimizer.anonymize(contenuto, doc_id=doc_id)
    risultati_multipli[doc_id] = {
        'testo_originale': contenuto,
        'testo_anonimizzato': testo_anonimizzato,
        'entità': entità
    }
    
    print(f"✅ Trovate {len(entità)} entità")
    
    # Mostra le prime 3 entità trovate
    for i, (placeholder, valore) in enumerate(list(entità.items())[:3]):
        tipo = placeholder.split('[')[1].split('_')[0]
        print(f"   🔸 {tipo}: {valore}")
    
    if len(entità) > 3:
        print(f"   ... e altre {len(entità) - 3} entità")

print(f"\n🎯 RISULTATI COMPLESSIVI:")
print("-" * 40)
print(f"📄 Documenti processati: {len(risultati_multipli)}")
print(f"🎯 Entità totali trovate: {sum(len(r['entità']) for r in risultati_multipli.values())}")


📚 Processamento documenti multipli...

📄 Processando: fattura_001
----------------------------------------
✅ Processato documento: 6 entità trovate
✅ Trovate 6 entità
   🔸 DATA: 12/01/2024
   🔸 IMPORTO: 1.250,00€
   🔸 INDIRIZZO: Via Garibaldi 45
   ... e altre 3 entità

📄 Processando: contratto_002
----------------------------------------
✅ Processato documento: 5 entità trovate
✅ Trovate 5 entità
   🔸 DATA: 22/08/1990
   🔸 IMPORTO: 2.500,00€
   🔸 INDIRIZZO: Corso Italia 88
   ... e altre 2 entità

📄 Processando:     preventivo_003
----------------------------------------
✅ Processato documento: 6 entità trovate
✅ Trovate 6 entità
   🔸 IMPORTO: 15.750,00€
   🔸 INDIRIZZO: Via Montenapoleone 12
   🔸 EMAIL: info@techsolutions.it
   ... e altre 3 entità

🎯 RISULTATI COMPLESSIVI:
----------------------------------------
📄 Documenti processati: 3
🎯 Entità totali trovate: 17


In [12]:
### 🎨 Visualizzazione Grafo Completo

# %%
# Mostra il riassunto completo del sistema
anonimizer.print_detailed_summary()

# Visualizza il grafo completo con tutti i documenti
print("\n🎨 Generazione visualizzazione grafo completo...")
anonimizer.visualize_graph(
    layout_type='spring',
    figsize=(18, 12),
    title="🕸️ Grafo Completo - Relazioni tra Documenti ed Entità"
)

# %% [markdown]
# ### 📊 Analisi Avanzata dei Risultati

# %%
# Crea un'analisi dettagliata dei tipi di entità più comuni
print("📊 ANALISI DETTAGLIATA DEI RISULTATI")
print("=" * 60)

stats = anonimizer.get_graph_stats()

# Crea un DataFrame per l'analisi
entità_data = []
for node, data in anonimizer.G.nodes(data=True):
    if data.get('tipo') not in ['Documento', 'Categoria']:
        entità_data.append({
            'ID': node,
            'Tipo': data.get('tipo', 'Unknown'),
            'Testo_Originale': data.get('testo_originale', 'N/A'),
            'Lunghezza': len(data.get('testo_originale', '')),
            'Timestamp': data.get('timestamp', 'N/A')
        })

df_entità = pd.DataFrame(entità_data)

if len(df_entità) > 0:
    print("\n📋 Top 5 tipi di entità più comuni:")
    tipo_counts = df_entità['Tipo'].value_counts()
    for i, (tipo, count) in enumerate(tipo_counts.head().items(), 1):
        emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "🏅"
        print(f"{emoji} {tipo}: {count} occorrenze")
    
    print(f"\n📏 Lunghezza media del testo per tipo:")
    lunghezza_media = df_entità.groupby('Tipo')['Lunghezza'].mean().sort_values(ascending=False)
    for tipo, media in lunghezza_media.head().items():
        print(f"   📐 {tipo}: {media:.1f} caratteri")
    
    # Visualizza distribuzione con un grafico a torta
    plt.figure(figsize=(12, 8))
    
    # Subplot 1: Grafico a torta
    plt.subplot(2, 2, 1)
    colors = [ENTITY_COLORS.get(tipo, ENTITY_COLORS['default']) for tipo in tipo_counts.index]
    plt.pie(tipo_counts.values, labels=tipo_counts.index, colors=colors, autopct='%1.1f%%')
    plt.title('📊 Distribuzione Tipi di Entità')
    
    # Subplot 2: Grafico a barre
    plt.subplot(2, 2, 2)
    bars = plt.bar(tipo_counts.index, tipo_counts.values, color=colors)
    plt.title('📈 Frequenza Tipi di Entità')
    plt.xticks(rotation=45)
    plt.ylabel('Numero di Occorrenze')
    
    # Aggiungi valori sulle barre
    for bar, value in zip(bars, tipo_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                str(value), ha='center', va='bottom', fontweight='bold')
    
    # Subplot 3: Lunghezza media
    plt.subplot(2, 2, 3)
    colors_len = [ENTITY_COLORS.get(tipo, ENTITY_COLORS['default']) for tipo in lunghezza_media.index]
    plt.barh(lunghezza_media.index, lunghezza_media.values, color=colors_len)
    plt.title('📏 Lunghezza Media Testo')
    plt.xlabel('Caratteri')
    
    # Subplot 4: Timeline (se disponibile)
    plt.subplot(2, 2, 4)
    if 'Timestamp' in df_entità.columns and df_entità['Timestamp'].notna().any():
        # Crea un grafico temporale semplificato
        plt.hist(range(len(df_entità)), bins=min(10, len(df_entità)), alpha=0.7, color='skyblue')
        plt.title('⏰ Timeline Processamento')
        plt.xlabel('Ordine di Processamento')
        plt.ylabel('Numero di Entità')
    else:
        plt.text(0.5, 0.5, 'Timeline non disponibile', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('⏰ Timeline Processamento')
    
    plt.tight_layout()
    plt.show()

else:
    print("⚠️ Nessuna entità trovata per l'analisi")


🔒 SISTEMA DI ANONIMIZZAZIONE - REPORT DETTAGLIATO

📊 STATISTICHE GENERALI
------------------------------
🔗 Nodi nel grafo: 27
🔗 Collegamenti: 39
📄 Documenti processati: 4
🎯 Entità totali trovate: 25
⏱️ Tempo elaborazione: 0.00 secondi

📋 DISTRIBUZIONE PER TIPO
------------------------------
📂 Categoria: 9
📄 Documento: 4
🎯 INDIRIZZO: 2
🎯 EMAIL: 2
🎯 IMPORTO: 2
🎯 PARTITA_IVA: 2
🎯 CODICE_FISCALE: 2
🎯 TELEFONO: 1
🎯 DATA: 1
🎯 IBAN: 1
🎯 CARTA_CREDITO: 1

📄 DETTAGLI ENTITÀ
------------------------------
   🔸 [DATA_0] (DATA)
      📝 Testo: 22/08/1990
      ⏰ Creato: 2025-07-03T15:49:43.134573
   🔸 DATA (Categoria)
      📝 Testo: N/A
   🔸 [INDIRIZZO_1] (INDIRIZZO)
      📝 Testo: Via Montenapoleone 12
      ⏰ Creato: 2025-07-03T15:49:43.134761
   🔸 INDIRIZZO (Categoria)
      📝 Testo: N/A
   🔸 [EMAIL_2] (EMAIL)
      📝 Testo: info@techsolutions.it
      ⏰ Creato: 2025-07-03T15:49:43.134775
   🔸 EMAIL (Categoria)
      📝 Testo: N/A
   🔸 [IBAN_3] (IBAN)
      📝 Testo: IT75B0300203280123456789012
 

In [11]:
### 🔍 Query Avanzate sul Grafo

# %%
print("🔍 QUERY AVANZATE SUL GRAFO")
print("=" * 50)

# Query 1: Trova documenti con più entità
print("\n📄 Documenti con il maggior numero di entità:")
documenti_con_entità = []
for node, data in anonimizer.G.nodes(data=True):
    if data.get('tipo') == 'Documento':
        num_entità = len(anonimizer.get_document_entities(node))
        documenti_con_entità.append((node, num_entità))

documenti_con_entità.sort(key=lambda x: x[1], reverse=True)
for doc, num in documenti_con_entità:
    print(f"   📋 {doc}: {num} entità")

# Query 2: Trova entità condivise tra documenti
print(f"\n🔗 Tipi di entità presenti in più documenti:")
tipi_per_documento = {}
for doc, num in documenti_con_entità:
    entità_doc = anonimizer.get_document_entities(doc)
    tipi_doc = set()
    for entità in entità_doc:
        if anonimizer.G.has_node(entità):
            tipo = anonimizer.G.nodes[entità].get('tipo', 'Unknown')
            tipi_doc.add(tipo)
    tipi_per_documento[doc] = tipi_doc

# Trova tipi comuni
tutti_i_tipi = set()
for tipi in tipi_per_documento.values():
    tutti_i_tipi.update(tipi)

for tipo in tutti_i_tipi:
    documenti_con_tipo = [doc for doc, tipi in tipi_per_documento.items() if tipo in tipi]
    if len(documenti_con_tipo) > 1:
        print(f"   🔸 {tipo}: presente in {len(documenti_con_tipo)} documenti")

# Query 3: Statistiche di centralità
print(f"\n🎯 Nodi più centrali (degree centrality):")
centrality = nx.degree_centrality(anonimizer.G)
sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

for node, score in sorted_centrality[:5]:
    node_type = anonimizer.G.nodes[node].get('tipo', 'Unknown')
    print(f"   🔸 {node} ({node_type}): {score:.3f}")

🔍 QUERY AVANZATE SUL GRAFO

📄 Documenti con il maggior numero di entità:


AttributeError: 'NERAnonimizer' object has no attribute 'get_document_entities'

In [ ]:
### 💾 Esportazione Risultati

# %%
print("💾 ESPORTAZIONE RISULTATI")
print("=" * 40)

# Esporta in diversi formati
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. Esporta in JSON
json_file = f"anonimizzazione_risultati_{timestamp}.json"
anonimizer.export_results(json_file, format='json')

# 2. Esporta in CSV
csv_file = f"entità_estratte_{timestamp}.csv"
anonimizer.export_results(csv_file, format='csv')

# 3. Esporta grafo per Gephi
gexf_file = f"grafo_entità_{timestamp}.gexf"
anonimizer.export_results(gexf_file, format='gexf')

# 4. Salva visualizzazione
img_file = f"grafo_visualizzazione_{timestamp}.png"
anonimizer.visualize_graph(
    layout_type='spring',
    figsize=(16, 12),
    save_path=img_file,
    title="🔒 Sistema di Anonimizzazione - Report Finale"
)

print(f"\n✅ Tutti i file sono stati esportati con timestamp: {timestamp}")

In [ ]:
### 🛠️ Funzioni di Utilità

# %%
def crea_report_markdown(anonimizer, filename=None):
    """
    📝 Crea un report markdown completo
    """
    if filename is None:
        filename = f"report_anonimizzazione_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    
    stats = anonimizer.get_graph_stats()
    
    report = f"""# 🔒 Report Sistema di Anonimizzazione

## 📊 Statistiche Generali
- **Nodi nel grafo**: {stats['numero_nodi']}
- **Collegamenti**: {stats['numero_collegamenti']}
- **Documenti processati**: {anonimizer.stats['documenti_processati']}
- **Entità totali**: {anonimizer.stats['entità_trovate']}
- **Tempo elaborazione**: {anonimizer.stats['tempo_elaborazione']:.2f} secondi

## 📋 Distribuzione per Tipo

"""
    
    for tipo, count in sorted(stats['entità_per_tipo'].items(), key=lambda x: x[1], reverse=True):
        report += f"- **{tipo}**: {count}\n"
    
    report += f"""
## 🎯 Entità Identificate

| ID | Tipo | Testo Originale | Timestamp |
|----|----|--------|-----------|
"""
    
    for node, data in anonimizer.G.nodes(data=True):
        if data.get('tipo') not in ['Documento', 'Categoria']:
            tipo = data.get('tipo', 'Unknown')
            testo = data.get('testo_originale', 'N/A')[:30] + "..." if len(data.get('testo_originale', '')) > 30 else data.get('testo_originale', 'N/A')
            timestamp = data.get('timestamp', 'N/A')
            report += f"| {node} | {tipo} | {testo} | {timestamp} |\n"
    
    report += f"""
## 🔗 Relazioni nel Grafo

"""
    
    for edge in anonimizer.G.edges(data=True):
        source, target, data = edge
        relation = data.get('relazione', 'unknown')
        report += f"- {source} **{relation}** {target}\n"
    
    # Salva il report
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(report)
    
    print(f"📝 Report markdown salvato: {filename}")
    return filename

def test_performance(anonimizer, num_tests=100):
    """
    ⚡ Test di performance
    """
    print("⚡ TEST DI PERFORMANCE")
    print("=" * 30)
    
    testi_test = [
        "Mario Rossi, email: mario@email.com, tel: 123456789",
        "Giulia Verdi, nata il 15/03/1990, CF: VRDGLI90C55H501Z",
        "Azienda XYZ, P.IVA: 12345678901, Via Roma 123",
        "Fattura 2024/001, importo: 1.500,00€, IBAN: IT60X0542811101000000123456"
    ]
    
    import time
    
    tempi = []
    for i in range(num_tests):
        testo = testi_test[i % len(testi_test)]
        
        start = time.time()
        anonimizer.anonymize(testo, doc_id=f"test_{i}")
        end = time.time()
        
        tempi.append(end - start)
    
    print(f"📊 Risultati test su {num_tests} documenti:")
    print(f"   ⏱️ Tempo medio: {np.mean(tempi):.4f} secondi")
    print(f"   ⚡ Tempo minimo: {np.min(tempi):.4f} secondi")
    print(f"   🐌 Tempo massimo: {np.max(tempi):.4f} secondi")
    print(f"   📏 Deviazione standard: {np.std(tempi):.4f} secondi")
    
    # Visualizza distribuzione tempi
    plt.figure(figsize=(10, 6))
    plt.hist(tempi, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('📊 Distribuzione Tempi di Elaborazione')
    plt.xlabel('Tempo (secondi)')
    plt.ylabel('Frequenza')
    plt.axvline(np.mean(tempi), color='red', linestyle='--', label=f'Media: {np.mean(tempi):.4f}s')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
### 🧪 Test Finali e Performance

# Genera report markdown
print("📝 Generazione report markdown...")
report_file = crea_report_markdown(anonimizer)

# Test di performance (versione ridotta per il notebook)
print("\n⚡ Avvio test di performance...")
test_performance(anonimizer, num_tests=50)

# %% [markdown]
# ### 🎉 Conclusioni e Prossimi Passi

# %%
print("🎉 CONCLUSIONI DEL SISTEMA DI ANONIMIZZAZIONE")
print("=" * 60)

stats_finali = anonimizer.get_graph_stats()

print(f"""
✅ SISTEMA COMPLETAMENTE OPERATIVO!

📊 RISULTATI FINALI:
   🔗 Nodi creati: {stats_finali['numero_nodi']}
   🔗 Relazioni mappate: {stats_finali['numero_collegamenti']}
   📄 Documenti processati: {anonimizer.stats['documenti_processati']}
   🎯 Entità identificate: {anonimizer.stats['entità_trovate']}
   ⏱️ Tempo totale: {anonimizer.stats['tempo_elaborazione']:.2f} secondi

🚀 FUNZIONALITÀ IMPLEMENTATE:
   ✅ Riconoscimento entità con regex personalizzate
   ✅ Supporto per modelli NER (opzionale)
   ✅ Creazione automatica di grafi delle relazioni
   ✅ Visualizzazioni avanzate con matplotlib
   ✅ Analisi statistiche dettagliate
   ✅ Esportazione in multipli formati (JSON, CSV, GEXF)
   ✅ Report automatici in Markdown
   ✅ Test di performance integrati

🔮 PROSSIMI PASSI SUGGERITI:
   🔸 Integrazione con interfaccia Streamlit
   🔸 Supporto per file PDF e DOCX
   🔸 Modelli NER personalizzati per domini specifici
   🔸 API REST per integrazione in sistemi esterni
   🔸 Dashboard interattiva con Plotly/Dash
   🔸 Backup automatico dei risultati
   🔸 Configurazione pattern regex via file YAML

💡 SUGGERIMENTI PER L'USO:
   - Usa use_ner=True per documenti con entità complesse
   - Personalizza i pattern regex in base al dominio
   - Esporta in GEXF per analisi avanzate con Gephi
   - Salva le visualizzazioni per presentazioni
   - Monitora le performance con test regolari
""")

print("\n🎯 Il sistema è pronto per l'uso in produzione!")
print("📚 Consulta la documentazione nelle celle markdown per dettagli aggiuntivi.")
